# **Introducción**
En este notebook vamos a implementar un modelo que detecte URLs maliciosas, para ello obtendremos un dataset de la siguiente dirección: https://raw.githubusercontent.com/saurabhsingh0/malacious-url-detector/master/model/datasets/malicious-url-detector_dataset.csv


---
Para abordar el problema he encontrado que se suelen utilizar 3 modelos diferentes:


*   *Naive Bayes Classifier*
*   *N-Gram with Naive Bayes Classifier*
*   *N-Gram with SVM Classifier*

He seleccionado el tercero por dos motivos:


1.   Naive Bayes ya lo hemos utilizado más veces en clase y además ya le he dado un sitio en el trabajo, a SVM no, así diversifico más y aprendo otros puntos de vista.
2.   He podido ver que el modelo de SVM funciona mejor para los datasets grandes como el nuestro.



**Importamos las dependencias**

In [1]:
import numpy as np
import random
import re
from nltk.util import ngrams
import itertools
import pandas as pd
from sklearn import svm
from joblib import dump, load
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

**Creamos los N-Grams**

Creamos las posibles combinaciones de N-Grams usando el alfabeto (inglés) en minúscula junto a los dígitos en un diccionario de Python

In [2]:
alphanum = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z','0','1','2','3','4','5','6','7','8','9']
permutations = itertools.product(alphanum, repeat=3)
featuresDict = {}
counter = 0
for perm in permutations:
    #print(perm)
    f=''
    for char in perm:
        f = f+char;
    featuresDict[(''.join(perm))] = counter
    counter = counter + 1

**Generamos los N-Gram**

Creamos una función que nos permita transformar un string en una lista de N-Gram

In [3]:
def generate_ngram(sentence):
    s = sentence.lower()
    s = ''.join(e for e in s if e.isalnum())
    processedList = []
    for tup in list(ngrams(s,3)):
        processedList.append((''.join(tup)))
    return processedList

**Preprocesado de los datos**

Creamos una función para preprocesar el dataset en formatos que nuestro modelo pueda aceptar y limpiar las URL de caracteres no deseados.

In [4]:
def preprocess_sentences(dataframe, X, y):
    #print(dataframe)
    for index,row in df.iterrows():
        url = row['url'].strip().replace("https://","")
        url = row['url'].strip().replace("http://","")
        url = url.replace("http://","")
        url = re.sub(r'\.[A-Za-z0-9]+/*','',url)
        for gram in generate_ngram(url):
            try:
                X[index][featuresDict[gram]] = X[index][featuresDict[gram]] + 1
            except:
                print(gram,"no existe")
        y[index] = int(row['label'])
    return (X,y)

**Gestión del dataset**

Cargamos el dataset y lo separamos en los conjuntos de train y test

In [5]:
url_dataframe = pd.read_csv("malicious-url-detector_dataset.csv")
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

**Gestión del modelo**

Vamos a utilizar SGDClassifier, el cual utiliza por defecto un SVM Lineal, que era el que queríamos utilizar, para el aprendizaje utilizaremos SGD (Stochastic Gradient Descent).

Utilizaremos un entrenamiento parcial.

In [6]:
no_of_rows = 5000
no_of_batches = int(train_df.shape[0]/no_of_rows) +1
classifier = SGDClassifier()
#print(no_of_batches)
for i in range(0, no_of_batches):
    start = no_of_rows*i
    if start + no_of_rows > train_df.shape[0] :
        df = train_df.iloc[start:,:]
    else :
        df = train_df.iloc[start:start+no_of_rows, :]
    df = df.reset_index()
    (X,y) = preprocess_sentences(df, \
                                 np.zeros([df.shape[0], 46656],dtype="int"), \
                                 np.zeros(df.shape[0],dtype="int"))
    classifier.partial_fit(X, y, classes=np.unique(y))

wnو no existe
nو² no existe
و²³ no existe
²³2 no existe
³2b no existe
e4ä no existe
4ää no existe
ääš no existe
äšç no existe
šçè no existe
çèç no existe
èçˆ no existe
çˆï no existe
ˆï¼ no existe
ï¼ˆ no existe
¼ˆå no existe
ˆåå no existe
ååº no existe
åºå no existe
ºåˆ no existe
åˆå no existe
ˆåï no existe
åï¼ no existe
ï¼3 no existe
¼32 no existe
azä no existe
zä½ no existe
ä½³ no existe
½³è no existe
³èƒ no existe
èƒ½ no existe
ƒ½ç no existe
½çå no existe
çåš no existe
åšæ no existe
šæå no existe
æåè no existe
åè½ no existe
è½ä no existe
½äc no existe
äca no existe
16ه no existe
6هé no existe
هéœ no existe
éœ² no existe
œ²ه no existe
²هه no existe
ههه no existe
ههچ no existe
هچˆ no existe
چˆه no existe
ˆهœ no existe
هœ6 no existe
œ60 no existe
18æ no existe
8æ³ no existe
æ³å no existe
³åœ no existe
åœæ no existe
œæœ no existe
æœº no existe
œº3 no existe
º32 no existe
azä no existe
zä½ no existe
ä½³ no existe
½³è no existe
³èƒ no existe
èƒ½ no existe
ƒ½ç no existe
½çå no existe
çåš no

**Realizamos el test del modelo**

In [7]:
no_of_rows = 5000
no_of_batches = int(test_df.shape[0]/no_of_rows) +1
#print(no_of_batches)
correct = 0;
incorrect = 0
for i in range(0, no_of_batches):
    start = no_of_rows*i
    if start + no_of_rows > train_df.shape[0] :
        df = train_df.iloc[start:,:]
    else :
        df = test_df.iloc[start:start+no_of_rows, :]
    df = df.reset_index()
    (X_test,y_test) = preprocess_sentences(df, \
                                 np.zeros([df.shape[0], 46656],dtype="int"), \
                                 np.zeros(df.shape[0],dtype="int"))
    y_pred = classifier.predict(X_test)
    for index,row in df.iterrows():
        if row['label'] == y_pred[index]:
            correct = correct+1
        else:
            incorrect = incorrect + 1   

reä no existe
eää no existe
ääš no existe
äšç no existe
šçˆ no existe
çˆå no existe
ˆåœ no existe
åœç no existe
œç¾ no existe
ç¾ž no existe
¾žç no existe
žçè no existe
çèç no existe
èçˆ no existe
çˆ3 no existe
ˆ32 no existe
reä no existe
eää no existe
ääš no existe
äšç no existe
šçˆ no existe
çˆå no existe
ˆåœ no existe
åœç no existe
œç¾ no existe
ç¾ž no existe
¾žç no existe
žçè no existe
çèç no existe
èçˆ no existe
çˆ3 no existe
ˆ32 no existe
wnا no existe
nاه no existe
اهء no existe
هءم no existe


**Comprobamos los resultados**

In [8]:
print("Predicciones correctas: ", correct)
print("Predicciones incorrectas: ", incorrect)
accuracy = (correct/test_df.shape[0])*100
print("Precisión: "'{0:.4g}'.format(accuracy), '%')

Predicciones correctas:  206679
Predicciones incorrectas:  3036
Precisión: 98.55 %


**Probamos con URLs a mano**

Creamos una funcion para preprocesar una única URL

In [9]:
def preprocess_sentences_url(url):
    X= np.zeros([1, 46656],dtype="int")
    url = url.strip().replace("https://","")
    url = url.replace("http://","")
    url = re.sub(r'\.[A-Za-z0-9]+/*','',url)
    for gram in generate_ngram(url):
        try:
            X[0][featuresDict[gram]] = X[0][featuresDict[gram]] + 1
            #print('preprocess_sentences')
            #print(X[index][featuresDict[gram]])
        except:
            print(gram,"no existe")
    return X        

Probamos a introducir una URL maliciosa

In [10]:
url = "www.itidea.it/centroesteticosothys/img/_notes/gum.exe"
test = preprocess_sentences_url(url)
pred = classifier.predict(test)
if pred == 1 :
    print(url, " es una URL maliciosa")
else:
    print(url, " es una URL segura")

www.itidea.it/centroesteticosothys/img/_notes/gum.exe  es una URL maliciosa


Probamos a introducir una URL segura

In [11]:
url = "https://u-tad.com/"
test = preprocess_sentences_url(url)
pred = classifier.predict(test)
if pred == 1 :
    print(url, " es una URL maliciosa")
else:
    print(url, " es una URL segura")

https://u-tad.com/  es una URL segura
